In [ ]:
import data as da
import collaborative_filtering as cf
import evaluation as ev
# Import Pandas
import pandas as pd

# Load restaurant data
movies = pd.read_csv('data/movies_subset.csv', low_memory=False)
# Load rating data
rating = pd.read_csv('data/ratings.csv', low_memory=False)
# Load test rating data
test_set = pd.read_csv('data/test.csv', low_memory=False)

In [1]:
class collaborative_filtering:
    def __init__(self):
        self.user_based_matrix = []
        self.item_based_metrix = []
        self.data_matrix = []
        self.data = None
    
    def create_pref_matrix(self, data):
        self.data = data
        matrix = []
        ratings, movies = data
        user_id_list = ratings.userId.drop_duplicates()
        movie_title_list = movies.title.drop_duplicates() # should't be dups here anyway just to be consistant
        movie_id_list = movies.movieId.drop_duplicates()
        user_prefs = {}
        for user, movie, rating in ratings.values:
            user = int(user)
            movie = int(movie)
            rating = float(rating)
            user_prefs[(user, movie)] = rating
        for user in user_id_list:
            u = []
            user = int(user)
            for movie in movie_id_list:
                movie = int(movie)
                if (user, movie) in user_prefs:
                    rating = user_prefs[(user, movie)]
                    u.append(rating)
                else:
                    u.append(np.nan)
            matrix.append(u)
        m = pd.DataFrame(matrix, columns=movie_title_list, index=user_id_list)
        return m

    def create_fake_user(self, rating):
        "*** YOUR   CODE HERE ***"
                                                                                        
        return rating

    def create_user_based_matrix(self, data):

        r, m = data
        user_id_list = r.userId.drop_duplicates()
        movie_title_list = m.title.drop_duplicates() # should't be dups here anyway just to be consistant
        movie_id_list = m.movieId.drop_duplicates()

        self.data_matrix = self.create_pref_matrix(data)
        m = self.data_matrix
        ratings = m.to_numpy()
        mean_user_rating = m.mean(axis=1).to_numpy().reshape(-1, 1)
        mean_user_rating.round(2)
        ratings_diff = ratings - mean_user_rating
        ratings_diff[np.isnan(ratings_diff)]=0
        ratings_diff.round(2)
        user_similarity = 1 - pairwise_distances(ratings_diff,metric='cosine')
        pred = mean_user_rating + user_similarity.dot(ratings_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T
        pred.round(2)
        self.user_based_matrix = pd.DataFrame(pred, columns=user_id_list, index=movie_title_list)

    def create_item_based_matrix(self, data):
        self.data_matrix = self.create_pref_matrix(data)
        m = self.data_matrix
        ratings = m.to_numpy()
        mean_user_rating = m.mean(axis=1).to_numpy().reshape(-1, 1)
        mean_user_rating.round(2)
        ratings_diff = (ratings - mean_user_rating)
        ratings_diff[np.isnan(ratings_diff)]=0
        ratings_diff.round(2)
        ratings_diff[np.isnan(ratings_diff)]=0
        item_similarity = 1-pairwise_distances(ratings_diff.T, metric='cosine')
        pred = mean_user_rating + ratings_diff.dot(item_similarity) / np.array([np.abs(item_similarity).sum(axis=1)])
        pred.round(2)
        self.item_based_metrix = pred

    def predict_movies(self, user_id, k, is_user_based=True):
        if is_user_based:
            print
            data_row_unrated_pred = np.asarray(self.user_based_matrix)[np.isnan(self.data_matrix.loc[int(user_id)])]
            idx = np.argsort(-data_row_unrated_pred)
            sim_scores = idx[0:k]
            return self.data[1]['Title'].iloc[sim_scores]

        sys.exit(1)


In [2]:

cf1 = cf.collaborative_filtering()

# PART 1 - DATA
def analsys(data):
    # da.watch_data_info(data)
    da.print_data(data)
    da.plot_data(data)

# PATR 2 - COLLABORATING FILLTERING RECOMMENDATION SYSTEM
def collaborative_filtering_rec(data, user_based = True):
    global cf1

    if(user_based):
        cf1.create_user_based_matrix(data)
    else:
        cf1.create_item_based_matrix(data)

    result = cf1.predict_movies("283225", 5)
    print(result)

# PART 3 - EVALUATION
def evaluate_rec():
    ev.precision_10(test_set, cf1)
    ev.ARHA(test_set,cf1)
    ev.RSME(test_set, cf1)

def main():
    #data
    # analsys((rating, movies))

    #collaborative filtering
    collaborative_filtering_rec((rating,movies))


    #evaluation
    # evaluate_rec()


if __name__ == "__main__":
    main()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 1344